In [2]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Topics rates

In [3]:
pd.read_hdf('../generated/topics_rates.hdf5.csv.9', stop=10).head()

,topic_id,platform,traffic_source,geo,topic_viewed
0,0,1,1,-1,20.310514
1,0,1,1,0,38.536388
2,0,1,1,1,4.740327
3,0,1,1,2,0.186612
4,0,1,1,4,0.037480


In [4]:
topics_rates = pd.read_hdf('../generated/topics_rates.hdf5.csv.9')
topics_rates = pd.DataFrame({'weight':topics_rates.groupby(['topic_id', 'platform'])['topic_viewed'].sum() }).reset_index()
topics_rates_platform = pd.DataFrame({'weight_p':topics_rates.groupby(['platform'])['weight'].sum()}).reset_index()
topics_rates = topics_rates.merge(topics_rates_platform, on=['platform'])
topics_rates['weight'] = topics_rates['weight'] / topics_rates['weight_p']  
topics_rates['weight_sqrt'] = np.sqrt(topics_rates['weight'])
topics_rates.drop(['weight_p'], axis=1, inplace=True)
topics_rates.head()

,topic_id,platform,weight,weight_sqrt
0,0,1,0.001353,0.036779
1,1,1,0.001870,0.043238
2,2,1,0.001545,0.039304
3,3,1,0.000711,0.026671
4,4,1,0.001119,0.033447


In [5]:
topics_rates.count()

topic_id       900
platform       900
weight         900
weight_sqrt    900
dtype: int64

In [6]:
sum(topics_rates['weight'])

2.9999999999999956

### Docs pairs

In [11]:
page_addoc_dist = pd.read_csv("../generated/page_addoc_dist.0.csv")
page_addoc_dist.count()

document_id    31437393
addoc_id       31437393
platform       31437393
dtype: int64

### Docs topics and norms

In [17]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(756510, 164476)

In [13]:
docs_used = pd.DataFrame({'document_id':page_addoc_dist.document_id.append(page_addoc_dist.addoc_id).unique()})
docs_used.count()

document_id    912081
dtype: int64

In [19]:
del page_addoc_dist

In [14]:
docs_topics = pd.read_csv("../download/documents_topics.csv")
docs_topics = docs_topics.merge(docs_used, on='document_id')
docs_topics.head()

,document_id,topic_id,confidence_level
0,1595802,140,0.073113
1,1595802,16,0.059416
2,1595802,143,0.045421
3,1595802,170,0.038867
4,1524246,113,0.196450


In [15]:
print len(docs_topics)

3801579


In [16]:
docs_topics = docs_topics.merge(topics_rates, on='topic_id', how='left').fillna(0)
docs_topics['value']=docs_topics['confidence_level']*docs_topics['weight_sqrt']
docs_topics['value2']=np.square(docs_topics['value'])
docs_topics.drop(['confidence_level','weight','weight_sqrt'], axis=1, inplace=True)
docs_topics.head()

,document_id,topic_id,platform,value,value2
0,1595802,140,1,0.018506,0.000342
1,1595802,140,2,0.016012,0.000256
2,1595802,140,3,0.016241,0.000264
3,1595802,16,1,0.008934,0.000080
4,1595802,16,2,0.010639,0.000113


In [22]:
print len(docs_topics)

11404737


In [18]:
docs_norms2 = pd.DataFrame({'norm2':docs_topics.groupby(['document_id', 'platform'])['value2'].sum()}).reset_index()
docs_topics.drop(['value2'], axis=1, inplace=True)
docs_norms2.head()

,document_id,platform,norm2
0,1,1,0.000012
1,1,2,0.000008
2,1,3,0.000011
3,2,1,0.000002
4,2,2,0.000003


### Distances

In [23]:
input_file = "../generated/page_addoc_dist.0.csv"
output_file = "../generated/final/page_addoc_topics_p_dist.csv"
chunksize=10**6
how='left'

In [24]:
first_time = True
size = 0
i = 0
    
f = open(output_file, 'w')

for page_addoc_dist in pd.read_csv(input_file, chunksize=chunksize):

    page_addoc_diffs = page_addoc_dist.merge(docs_topics, left_on=['addoc_id', 'platform'], right_on=['document_id', 'platform'])
    page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
    page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
    page_addoc_diffs = page_addoc_diffs.merge(docs_topics, on=['document_id', 'topic_id', 'platform'])
    page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
    page_addoc_diffs.drop(['topic_id', 'value_x', 'value_y'], axis=1, inplace=True)
    page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id', 'platform'])['value_2xy'].sum()}).reset_index()

    page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id','platform'])
    page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id','platform'], right_on=['document_id','platform'])
    page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
    page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
    page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id', 'platform'], how='left').fillna(0)
    page_addoc_dist['dist'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
    page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

    if first_time:
        page_addoc_dist.to_csv(f, index = False, header=True)
        f.close()
        f = open(output_file, 'a')
        first_time = False
    else:
        page_addoc_dist.to_csv(f, index = False, header=False)
    
    size+=chunksize
    i+=1
    print i, size
            
print 'Done'
    
f.close()

1 1000000
2 2000000
3 3000000
4 4000000
5 5000000
6 6000000
7 7000000
8 8000000
9 9000000
10 10000000
11 11000000
12 12000000
13 13000000
14 14000000
15 15000000
16 16000000
17 17000000
18 18000000
19 19000000
20 20000000
21 21000000
22 22000000
23 23000000
24 24000000
25 25000000
26 26000000
27 27000000
28 28000000
29 29000000
30 30000000
31 31000000
32 32000000
Done


In [25]:
pd.read_csv("../generated/final/page_addoc_topics_p_dist.csv", nrows=5).head()

,document_id,addoc_id,platform,dist
0,2,1489042,2.0,0.033707
1,452,1489042,2.0,0.033986
2,515,1489042,2.0,0.033698
3,1608,1489042,2.0,0.033781
4,3260,1489042,2.0,0.034061
